In [2]:
import cv2
import kagglehub
from ultralytics import YOLO
import os

In [3]:
# download playing card dataset using kagglehub
data_path = kagglehub.dataset_download("andy8744/playing-cards-object-detection-dataset")
os.listdir(data_path)

['data.yaml',
 'kaggle_data.yaml',
 'README.roboflow.txt',
 'test',
 'train',
 'valid']

In [4]:
# create and train YOLO model for playing card detection
model = YOLO("yolo11n.yaml")
results = model.train(data=data_path+"/data.yaml", epochs=3)

Ultralytics 8.3.111  Python-3.13.3 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.yaml, data=C:\Users\Mark\.cache\kagglehub\datasets\andy8744\playing-cards-object-detection-dataset\versions\4/data.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fals

train: Scanning C:\Users\Mark\.cache\kagglehub\datasets\andy8744\playing-cards-object-detection-dataset\versions\4\train\labels.cache... 14000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 14000/14000 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.1 ms, read: 163.235.1 MB/s, size: 27.9 KB)


val: Scanning C:\Users\Mark\.cache\kagglehub\datasets\andy8744\playing-cards-object-detection-dataset\versions\4\valid\labels.cache... 4000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4000/4000 [00:00<?, ?it/s]


Plotting labels to runs\detect\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000179, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train2
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       4000      15159    0.00696      0.543    0.00811    0.00381

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        3/3      2.99G      2.082      4.293      1.608         68        640: 100%|██████████| 875/875 [01:21<00:00, 10.67it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 125/125 [00:16<00:00,  7.41it/s]


                   all       4000      15159    0.00966      0.764     0.0129    0.00782

3 epochs completed in 0.084 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 5.5MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics 8.3.111  Python-3.13.3 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
YOLO11n summary (fused): 100 layers, 2,592,292 parameters, 0 gradients, 6.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 125/125 [00:16<00:00,  7.78it/s]


                   all       4000      15159    0.00967      0.764     0.0129    0.00782
                   10c        180        282    0.00965      0.798     0.0148    0.00812
                   10d        188        290     0.0106      0.803     0.0135    0.00869
                   10h        201        322     0.0117      0.776      0.017     0.0104
                   10s        165        250    0.00784      0.948     0.0129    0.00747
                    2c        193        295    0.00933      0.851     0.0105    0.00561
                    2d        167        259    0.00814      0.865     0.0107    0.00654
                    2h        187        294    0.00897       0.68    0.00843    0.00516
                    2s        192        299    0.00944      0.692     0.0097    0.00566
                    3c        201        308    0.00893      0.782    0.00873    0.00565
                    3d        167        250    0.00757      0.892    0.00896    0.00577
                    3

In [3]:
# read a frame from the webcam to get image of cards
cam = cv2.VideoCapture(0)
_, frame = cam.read()
cam.release()

In [4]:
# show the image
cv2.imshow("pokervision", frame)

# close the window on any keypress
if cv2.waitKey(0) != -1:
	cv2.destroyAllWindows()